In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join Example 11
val model = Network("model")

// Create network
val delay = Delay(model, "Delay1")
val delay2 = Delay(model, "Delay2")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)

val jobclass1 = ClosedClass(model, "class1", 10, delay, 0)

In [ ]:
// Service configurations
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(1.0))
delay.setService(jobclass1, Exp(0.5))
delay2.setService(jobclass1, Exp(2.0))

In [ ]:
// Set routing matrix
val P = model.initRoutingMatrix()

// Serial routing through delays: delay -> delay2 -> fork
P.set(jobclass1, jobclass1, delay, delay2, 1.0)
P.set(jobclass1, jobclass1, delay2, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = arrayOf(
    JMT(model, "seed", 23000),
    MVA(model)
)

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This fork-join network with serial delays demonstrates:

1. **Pre-Fork Processing**:
   - Jobs undergo sequential processing before parallelization
   - **Delay1**: Slow pre-processing (rate = 0.5)
   - **Delay2**: Fast pre-processing (rate = 2.0)
   - Creates a pipeline: Delay1 → Delay2 → Fork

2. **Mixed Serial-Parallel Structure**:
   - Serial stage: Delay1 → Delay2 (sequential processing)
   - Parallel stage: Fork → {Queue1, Queue2} → Join (parallel processing)
   - Return stage: Join → Delay1 (cycle completion)

3. **Service Rate Configuration**:
   - **Delay1**: Bottleneck (rate = 0.5, slowest)
   - **Delay2**: Fast (rate = 2.0, fastest)
   - **Queue1, Queue2**: Moderate (rate = 1.0 each)

4. **Performance Characteristics**:
   - Delay1 likely becomes the system bottleneck
   - Queue buildup primarily at the slowest stage
   - Fork-join synchronization adds complexity
   - Different phases have different resource utilization

5. **Workflow Pattern**:
   - **Phase 1**: Sequential pre-processing (preparation)
   - **Phase 2**: Parallel processing (main work)
   - **Phase 3**: Synchronization and completion
   - **Phase 4**: Return to start (cycle)

6. **Solver Comparison**:
   - JMT: Simulation with detailed timing
   - MVA: Analytical approximation for fork-join

The model represents:
- Manufacturing with preparation → parallel work → assembly
- Software workflows with setup → parallel execution → merge
- Service systems with screening → parallel service → coordination
- Data processing with validation → parallel analysis → aggregation

Key insight: Serial delays before fork-join can significantly impact overall system performance, with the slowest serial stage often becoming the primary bottleneck regardless of parallel stage efficiency.